In [1]:
#Importando todas as bibliotecas necessárias
import pandas as pd
import numpy as np 
from pandas.plotting import scatter_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import metrics


import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
 

In [ ]:
'''#Carregando o arquivo com a biblioteca do pandas que está guardado na variável pd
dataSetLOL = 'E://ESTUDANDO//TCC//_CAPITULO_4_LOL//LOL//NOVO_LOL//games.csv'
lol = pd.read_csv(dataSetLOL, sep =',')

#Função necessária para mostrar todas as linhas e colunas do dataset sem truncá-las
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#Exibe as 5 primeiras linhas do dataset para fins de verificação
lol.head()'''

In [2]:
#Carregando o arquivo com a biblioteca do pandas que está guardado na variável pd
dataSetLOL = 'E://ESTUDANDO//TCC//_CAPITULO_4_LOL//LOL//LeagueOfLedends2017.xlsx'
lol = pd.read_excel(dataSetLOL, encoding='latin1')

#Função necessária para mostrar todas as linhas e colunas do dataset sem truncá-las
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



In [ ]:
#Exibe as 5 primeiras linhas do dataset para fins de verificação
lol.head(30)

In [ ]:
#Segundo a função shape, temos 37404 linhas e 98 colunas no dataset
lol.shape

In [ ]:
#lol['player'].value_counts()

In [ ]:
'''
Aqui cada time é transformado em uma linha para depois descobrirmos qual a quantidade de times
da temporada.
'''
#lol = lol.groupby('team').nunique()


In [ ]:
#lol

In [ ]:
#Aqui é contado a quantidade de times do ano de 2017 de partidas profissionais.
#Temos ao total 116 times
count = 0
for index, row in lol.iterrows():
    count +=1
print(count)

<h1>Chances de vitória: Variáveis de interesse</h1>
<p><h3>Queremos checar quais times venceram a partida - "result" = 1 na tabela -, usando como parâmetro, variáveis de interesse.</h3></p>
<p>As variáveis de interesse para a análise das chances de vitória para esse caso de uso são:</p> 
<ul>
    <li><b>gameID:        </b> Identificador da partida, único para cada servidor/liga.</li>
    <li><b>side:          </b> Lado em que o time analisado começou na partida (Azul ou Vermelho).</li>
    <li><b>champion:      </b> Campeões escolhidos na partida pelo time.</li>
    <li><b>position:      </b> Posição em que os campeões jogam na partida.</li>
    <li><b>team:          </b> Nome do time na partida.</li>
</ul>
    <p>Entretando, para utilizar as variáveis para a criação de um modelo preditivo, é necessário primeiramente, trabalhá-las adequadamente.</p>

<h1>Funcionamento das partidas:</h1>
<p>Devido a natureza de como se dá o funcionamento das partidas de League of Legends, com 5 jogadores para cada um dos dois times, cada jogar escolhendo seu campeão e sua posição na partida, colunas novas serão criadas em um novo dataframe identificando qual campeão cada jogador dos dois times pegou na partida e qual posição cada um pegou.</p>

<h3>Criando funções de padronização</h3>
<p>Abaixo serão criadas funções cujo objetivo será padronizar as colunas "team" e "champion" que provavelmente possui muitos valores inválidos</p>

In [3]:
'''
Aqui todos os campeões estão sendo adicionados em uma lista de palavras e as mesmas estão sendo limpas:
todos os caracteres minúsculos, eliminando todas os espaços em branco, eliminando todas as pontuações,
removendo todas as palavras duplicadas ( com o dict). No final, todas as palavras modificadas serão adicionados em 
uma lista de campeões. Essa limpeza será uma nova funcionalidade criada em uma função chamada "padronizar_campeoes"
'''
import string

def padronizar_campeoes(champions):
    champions = champions.lower().translate(str.maketrans('', '', string.punctuation)).replace(' ', '').replace('-', '').strip('')
    return champions

In [4]:
'''
Aqui todos os times estão sendo adicionados em uma lista de palavras e as mesmas estão sendo limpas:
todos os caracteres minúsculos, eliminando todas os espaços em branco, eliminando todas as pontuações,
removendo todas as palavras duplicadas ( com o dict). No final, todas as palavras modificadas serão adicionados em 
uma lista de times. Essa limpeza será uma nova funcionalidade criada em uma função chamada "padronizar_times"
'''
import string

def padronizar_times(teams):
    teams = teams.lower().translate(str.maketrans('', '', string.punctuation)).replace(' ', '').replace('-', '').strip('')
    return teams

<h1>Aplicando as funções linha a linha:</h1>
<p>O objetivo agora é aplicar essas funções linha a linha, passando a variável que queremos alterar como parâmetro. Para fazer isso,
usarei o método "map", passando como argumento o nome da função e atribuindo isso a uma nova coluna que será criada com os 
dados alterados. As novas colunas criadas terão o mesmo nome da coluna original, mas o o 's' na frente de standard</p>

In [5]:
'''
Aplicando a função de padronização na coluna "champion". Uma nova coluna será criada e 
já padronizada, chamada "s_champion"
'''
lol["s_champion"] = lol["champion"].map(padronizar_campeoes)


In [6]:
'''
Aplicando a função de padronização na coluna "team". Uma nova coluna será criada e 
já padronizada, chamada "s_team"
'''
lol["s_team"] = lol["team"].map(padronizar_times)


In [7]:
lol.head()

,gameid,url,league,split,date,week,game,patchno,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,k,d,a,teamkills,teamdeaths,doubles,triples,quadras,pentas,fb,fbassist,fbvictim,fbtime,kpm,okpm,ckpm,fd,fdtime,teamdragkills,oppdragkills,elementals,oppelementals,firedrakes,waterdrakes,earthdrakes,airdrakes,elders,oppelders,herald,heraldtime,ft,fttime,firstmidouter,firsttothreetowers,teamtowerkills,opptowerkills,fbaron,fbarontime,teambaronkills,oppbaronkills,dmgtochamps,dmgtochampsperminute,dmgshare,earnedgoldshare,wards,wpm,wardshare,wardkills,wcpm,visionwards,visionwardbuys,visiblewardclearrate,invisiblewardclearrate,totalgold,earnedgpm,goldspent,gspd,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15,xpat10,oppxpat10,xpdat10,csat10,oppcsat10,csdat10,csat15,oppcsat15,csdat15,s_champion,s_team
0,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,1,Blue,Top,Lourlo,Team Liquid,Gnar,LeBlanc,Zac,Shen,Tristana,Kog'Maw,25.383333,0,0,2,0,3,18,0,0,0,0,0,0.0,0,6.73213,0,0.118188,0.827315,0,11.9418,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,12.368267,0.0,0.0,4,10,0.0,21.536467,0.0,1.0,8725.0,343.729481,0.269631,0.239989,14.0,0.551543,0.184211,2.0,0.078792,5,5.0,,,9067.0,224.749048,9025.0,0.028090,231.0,4.0,3.0,0.0,9.258043,3378.0,3264.0,114.0,5542.0,5419.0,123.0,4669.0,4677.0,-8.0,94.0,89.0,5.0,150.0,136.0,14.0,gnar,teamliquid
1,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,2,Blue,Jungle,Dardoch,Team Liquid,Gragas,LeBlanc,Zac,Shen,Tristana,Kog'Maw,25.383333,0,0,5,1,3,18,0,0,0,0,0,1.0,0,6.73213,0,0.0787919,0.827315,0,11.9418,0,2,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,12.368267,0.0,0.0,4,10,0.0,21.536467,0.0,1.0,4300.0,169.402495,0.132884,0.176005,15.0,0.590939,0.197368,3.0,0.118188,5,6.0,,,7546.0,164.827840,6650.0,-0.241322,15.0,98.0,58.0,5.0,4.451740,2936.0,2883.0,53.0,4467.0,5223.0,-756.0,3134.0,3161.0,-27.0,47.0,47.0,0.0,62.0,79.0,-17.0,gragas,teamliquid
2,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,3,Blue,Middle,Mickey,Team Liquid,Ekko,LeBlanc,Zac,Shen,Tristana,Kog'Maw,25.383333,0,0,3,0,3,18,0,0,0,0,0,0.0,0,6.73213,0,0.315167,0.827315,0,11.9418,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,12.368267,0.0,0.0,4,10,0.0,21.536467,0.0,1.0,6698.0,263.873933,0.20699,0.257531,16.0,0.630335,0.210526,4.0,0.157584,6,6.0,,,9484.0,241.177150,8700.0,-0.171789,247.0,2.0,1.0,0.0,9.809586,3446.0,3468.0,-22.0,5236.0,6286.0,-1050.0,5088.0,5169.0,-81.0,93.0,92.0,1.0,142.0,143.0,-1.0,ekko,teamliquid
3,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,4,Blue,ADC,Piglet,Team Liquid,Vayne,LeBlanc,Zac,Shen,Tristana,Kog'Maw,25.383333,0,0,3,1,3,18,0,0,0,0,0,1.0,0,6.73213,0,0.157584,0.827315,0,11.9418,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,12.368267,0.0,0.0,4,10,0.0,21.536467,0.0,1.0,10367.0,408.417597,0.320375,0.197459,10.0,0.393959,0.131579,1.0,0.039396,2,3.0,,,8056.0,184.919764,8275.0,-0.191257,186.0,1.0,1.0,0.0,7.367039,2933.0,3287.0,-354.0,4578.0,5635.0,-1057.0,3141.0,2984.0,157.0,68.0,75.0,-7.0,112.0,128.0,-16.0,vayne,teamliquid
4,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,5,Blue,Support,Matt,Team Liquid,Lulu,LeBlanc,Zac,Shen,Tristana,Kog'Maw,25.383333,0,3,5,0,3,18,1,0,0,0,1,0.0,0,6.73213,0.118188,0.0393959,0.827315,0,11.9418,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,12.368267,0.0,0.0,4,10,0.0,21.536467,0.0,1.0,2269.0,89.389363,0.0701196,0.129016,21.0,0.827315,0.276316,8.0,0.315167,6,7.0,,,6429.0,120.822587,6880.0,-0.092862,12.0,0.0,0.0,0.0,0.472751,2256.0,2389.0,-133.0,3184.0,4298.0,-1114.0,2593.0,2816.0,-223.0,5.0,2.0,3.0,7.0,2.0,5.0,lulu,teamliquid


<h3>Criando um novo dataframe</h3>
<p>Para facilitar na utilização dos modelos preditivos de Machine Learning e na economia de memória, não precisamos das quase 100 colunas do dataset. Portanto, um novo dataframe será criado contendo apenas as colunas necessárias.</p>

In [7]:
'''
Criando uma nova variável contendo somente as variáveis de interesse
'''

w_lol = pd.concat([ lol["result"], lol["side"], lol["s_champion"], 
                   lol["position"], lol["s_team"], lol["gameid"]], axis = 1).copy()


In [8]:
w_lol.head(30)

,result,side,s_champion,position,s_team,gameid
0,0,Blue,gnar,Top,teamliquid,1002300127
1,0,Blue,gragas,Jungle,teamliquid,1002300127
2,0,Blue,ekko,Middle,teamliquid,1002300127
3,0,Blue,vayne,ADC,teamliquid,1002300127
4,0,Blue,lulu,Support,teamliquid,1002300127
5,1,Red,jarvaniv,Top,dignitas,1002300127
6,1,Red,maokai,Jungle,dignitas,1002300127
7,1,Red,taliyah,Middle,dignitas,1002300127
8,1,Red,varus,ADC,dignitas,1002300127
9,1,Red,janna,Support,dignitas,1002300127


<h3>Padronizando valores em strings </h3>
<p> Os modelos preditivos aceitam apenas valores numéricos. Portanto, as variáveis de interesse cujos valores são strings precisam ser padronizadas. Existem 3 variáveis em que esses problemas ocorrem: 'champion','side', 'position'. Para realizar esse processo de padronização, uma função foi criada com o objetivo de utilizar o método "LabelEncoder" pertencente a biblioteca sklearn.preprocessing. Esse método foi explicitado no site "StackOverflow" por um usuário, no seguinte link: https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn <p>

In [9]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

class PadronizacaoStrings:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self 

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [ ]:
colunasStrings = ['side', 'team', 'champion1', 'position1', 'champion2', 'position2',
                  'champion3', 'position3', 'champion4', 'position4',
                  'champion5', 'position5', 'enemy_champion1', 'enemy_position1',
                  'enemy_champion2', 'enemy_position2', 'enemy_champion3', 'enemy_position3',
                  'enemy_champion4', 'enemy_position4', 'enemy_champion5', 'enemy_position5']

In [10]:
'''
Aqui, a função de padronização de Strings será chamado e esse processo será atribuído a um novo
dataframe cujo nome é "s_lol_ML"
'''
s_lol = (PadronizacaoStrings(columns = ['side', 's_champion', 'position', 's_team']).fit_transform(w_lol)).copy()


<h3>Mudando novamente o dataframe para a forma final</h3>
<p>Aqui acontece a mudança mais essencial ao dataframe. Atualmente ele está estruturado da seguinte forma: Cada jogo possui o seu id único (gameid). Para cada jogo, temos listados 5 jogadores por linha, para cada um dos dois times. Logo depois, temos mais duas linhas listando os dois times participantes da partida. Ou seja, para cada partida, temos um total de 12 linhas representando cada jogador, seus times e suas posições na partida. A mudança feita no dataframe foi alocar as informações contidas nessas 12 linhas para uma única linha, aumentando o número de colunas de 6 para 24</p>

In [11]:
'''
Aqui acontece a transformação mais importante do dataframe. 
'''
i = 0

result = {}

for index, value in s_lol.iterrows():
    if (not(value.gameid in result)):
        result[value.gameid] = {
            "result": value.result,
            "side": value.side,
            "team": value.s_team,
            "gameid": value.gameid
        }
        i = 0
    if i == 10:
        continue
    
    if (i < 5):
        result[value.gameid]["champion"+str(i+1)] = value.s_champion
        result[value.gameid]["position"+str(i+1)] = value.position
    else:
        result[value.gameid]["enemy_champion"+str(i-4)] = value.s_champion
        result[value.gameid]["enemy_position"+str(i-4)] = value.position
    
    i = i + 1

    
df_lol = pd.DataFrame.from_dict(result, orient='index')


    


In [12]:
s_lol.head(30)

,result,side,s_champion,position,s_team,gameid
0,0,0,32,5,96,1002300127
1,0,0,33,1,96,1002300127
2,0,0,23,2,96,1002300127
3,0,0,113,0,96,1002300127
4,0,0,63,3,96,1002300127
5,1,1,41,5,17,1002300127
6,1,1,67,1,17,1002300127
7,1,1,102,2,17,1002300127
8,1,1,112,0,17,1002300127
9,1,1,40,3,17,1002300127


In [ ]:
df_lol

In [ ]:
'''
Teste para verificar se o novo dataset padronizado alocou os valores certos para cada partida. 
'''
for index, row in lol.iterrows():
    if(row.gameid == 200035):
        print(row.s_champion)

<h3>Mudança de dimensionalidade:</h3>
    <p>Como era esperado, a dimensionalidade mudou drasticamente. Isso ocorre porque todas as linhas foram divididas em vários pedaços diferentes. E esses pedaços foram todos alocados em 20 novas colunas que representa cada posição e cada campeão escolhido por cada um dos 10 jogadores ( dos dois times) da partida. </p>

In [13]:
w_lol.shape

(37404, 6)

In [14]:
df_lol.shape

(3090, 24)

<h3>Verificando os tipos das variáveis:</h3>
    <p>Queremos apenas variáveis inteiras ( ou float) para poder usá-las nos modelos preditivos.</p>

In [15]:
'''
Como pode ser visto abaixo, a variável "gameid" ( que representa uma coluna do dataframe), está
representada como uma string. Precisamos tratá-la. Entretanto, não iremos mais precisar dessa coluna.
Então vamos apenas descartá-la com o método "drop" do Pandas.
'''
df_lol.dtypes

result              int64
side                int64
team                int64
gameid             object
champion1           int64
position1           int64
champion2           int64
position2           int64
champion3           int64
position3           int64
champion4           int64
position4           int64
champion5           int64
position5           int64
enemy_champion1     int64
enemy_position1     int64
enemy_champion2     int64
enemy_position2     int64
enemy_champion3     int64
enemy_position3     int64
enemy_champion4     int64
enemy_position4     int64
enemy_champion5     int64
enemy_position5     int64
dtype: object

In [16]:
'''
Deletando a coluna "gameid" do dataframe.
'''
del df_lol['gameid']

In [17]:
df_lol.dtypes

result             int64
side               int64
team               int64
champion1          int64
position1          int64
champion2          int64
position2          int64
champion3          int64
position3          int64
champion4          int64
position4          int64
champion5          int64
position5          int64
enemy_champion1    int64
enemy_position1    int64
enemy_champion2    int64
enemy_position2    int64
enemy_champion3    int64
enemy_position3    int64
enemy_champion4    int64
enemy_position4    int64
enemy_champion5    int64
enemy_position5    int64
dtype: object

<h3>Verificando a existência de dados nulos</h3>

In [18]:
#Verificando a quantidade de dados nulos em cada coluna
df_lol.isnull().sum()

result             0
side               0
team               0
champion1          0
position1          0
champion2          0
position2          0
champion3          0
position3          0
champion4          0
position4          0
champion5          0
position5          0
enemy_champion1    0
enemy_position1    0
enemy_champion2    0
enemy_position2    0
enemy_champion3    0
enemy_position3    0
enemy_champion4    0
enemy_position4    0
enemy_champion5    0
enemy_position5    0
dtype: int64

<h3>Dividindo o Dataframe: </h3>
<p>A divisão feita pelo train_test_split é aleatória mas é feito somente uma única vez. Para garantir a aleatoriedade do processo e a separação igualitária do dataframe em treino e teste, podemos dividir o dataframe em pecaços iguais ("folds"), embaralhar cada pedaço ("shuffle") e aplicar a divisão em cada pedaço. Para fazer isso, precisamos usar o método "Kfold" pertencente a biblioteca "sklearn.model_selection"</p>

<h3>Calculando a acurácia:</h3>
<p>Após as variáveis de interesse serem treinadas, elas serão usadas para tentar prever a vitória da partida comparando com as variáveis de teste. A acurácia, medirá a taxa de acerto das variáveis de interesse na previsão sobre as chances de vitória da partida. O cálculo da acurácia será feito por cada divisão feita pelo método kfold. No final, será calculado a <b>média da acurácia de todas as  divisões do dataframe feita pelo método kfold</b>. Essa será a acurácia final, tendo somente a variável de interesse calculada no momento como parâmetro determinístico para a vitória da partida. </p>

In [19]:
from sklearn.model_selection import KFold


<h3>Testando múltiplos modelos preditivos em função de sua performance: </h3>
<p>Antes de realizar os processos de cálculo das chances de vitória em função das variáveis de interesse através dos modelos preditivos, primeiramente é necessário definir qual é o modelo preditivo que possui melhor perfomance para a construção de análises preditivas. 6 modelos serão analisados com todas as variáveis possíveis. Entretanto, no final, somente um modelo será usado para todas as variáveis. São eles: LOGISTIC_REGRESSION, LINEAR_DISCRIMINATION_ANALISYS, KNEIGHBORS_CLASSIFIER, RANDOM_FORES_CLASSIFIER, DECISION_TREE_CLASSIFIER, GAUSSIANA. </p>

<h3>Cross Validation Score NÃO treina (fit) o dataset para futuras previsões</h3>
<p>
    De acordo com uma dúvida publicada no Stack Overflow (https://stackoverflow.com/questions/42263915/using-sklearn-cross-val-score-and-kfolds-to-fit-and-help-predict-model?rq=1) o método cross_validation_score serve apenas estimar quais modelos predivitos possuem a melhor performance. Isso é feito a partir da subdivisão do dataset de treino ( o dataset de teste original não é trabalhado), em vários folds ( pedaços) onde 1 dos pedaços será usado como teste. Isso será feito k vezes ( k é passado como parâmetro), sendo que em cada iteração, 1 novo pedaço do dataset de treino será escolhido para testar a acurácia do mesmo. No final, a média da acurácia das k iterações será feita e o resultado da acurácia será estimado.  O dataset de teste original não é usado é porque o objetivo é generalizar a previsão para análises futuras. Não queremos que o algoritmo decore os resultados do dataset de teste. .</p>

In [20]:
df_lol.head()

,result,side,team,champion1,position1,champion2,position2,champion3,position3,champion4,position4,champion5,position5,enemy_champion1,enemy_position1,enemy_champion2,enemy_position2,enemy_champion3,enemy_position3,enemy_champion4,enemy_position4,enemy_champion5,enemy_position5
200035,1,0,6,91,5,126,1,102,2,110,0,106,3,30,5,33,1,100,2,15,0,104,3
200036,1,0,22,41,5,24,1,58,2,8,0,101,3,33,5,59,1,117,2,112,0,106,3
200039,0,0,38,91,5,24,1,100,2,8,0,47,3,99,5,59,1,17,2,15,0,130,3
200048,0,0,9,28,5,77,1,58,2,44,0,47,3,33,5,54,1,78,2,15,0,106,3
200050,1,0,100,30,5,24,1,49,2,15,0,12,3,91,5,59,1,58,2,8,0,130,3


In [21]:
'''
A variável X recebe todas as linhas do dataframe e todas as colunas ( menos a coluna alvo "result").
A variável y recebe somente a coluna "result"
'''

X = np.array(df_lol.iloc[:,1:])
y = np.array(df_lol['result'])

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn import model_selection

import warnings
warnings.filterwarnings('ignore')


'''
Aqui os modelos preditivos estão sendo testados em função de sua performance EM TODAS AS VARIÁVES
DE INTERESSE. No final, somente 1 modelo preditivo será escolhido para ser usado. O método utilizado
é dividir o dataset em 10 pedaços e calcular a acurácia de cada pedaço. No final calcularemos média 
da acurácia desses 10 pedaços. Quanto maior a média final, melhor o modelo preditivo. Importante 
salientar que o treinamento não está sendo feito aqui. Apenas estimativas de quais modelos funcionam
melhor. 

'''

models = []
models.append(('RANDOM_FOREST_CLASSIFIER', RandomForestClassifier(n_estimators=500, random_state = 0)))
models.append(('LOGISTIC_REGRESSION', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LINEAR_DISCRIMINATION_ANALISYS', LinearDiscriminantAnalysis()))
models.append(('KNEIGHBORS_CLASSIFIER', KNeighborsClassifier()))
models.append(('DECISION_TREE_CLASSIFIER', DecisionTreeClassifier()))
models.append(('NAIVE_BAYS', GaussianNB()))
#models.append(('RIDGE', Ridge()))
#models.append(('STANDARD_SCALLER', StandardScaler()))
#models.append(('DECISION_TREE_REGRESSOR',DecisionTreeRegressor(max_depth=8, min_samples_leaf=0.13, random_state=3)))
models.append(('SVM', SVC(gamma='auto')))



# evaluate each model in turn
results = []
names = []
plota_names = []
plota_acuracia = []



for name, model in models:
    kf = KFold(10, shuffle=True, random_state = 1)
    hit_rate = 0
    for training_lines, testing_lines in kf.split(X,y):
       
        kf = model_selection.KFold(10, shuffle=True, random_state = 1)
        X_train, X_test = X[training_lines], X[testing_lines]
        y_train, y_test = y[training_lines], y[testing_lines]

        cv_results = cross_val_score(model, X_train, y_train, cv=kf, scoring='accuracy')
        results.append(cv_results)
        names.append(name)

        print('%s => Accuracy: %f, Standard Deviation: (%f)' % (name, cv_results.mean(), cv_results.std()))

        hit_rate += cv_results.mean()
        
    #plota_names.append(names)
    #plota_acuracia.append(hit_rate/10)

    final_accuracy = hit_rate/10   
    print("Média das 10 acurácias: ",final_accuracy)
    #print("Desvio padrão da média final :",hit_rate.std())
    print()

# Compare Algorithms


'''fig = plt.figure(figsize = (12,8))
fig.suptitle("Comparação de algoritmos")
ax = fig.add_subplot(111)
sns.barplot( x=plota_names, y=final_accuracy, data=df_lol, color="b")
ax.set_xticklabels(plota_names)
ax.set_yticklabels(final_accuracy)
plt.show()'''




RANDOM_FOREST_CLASSIFIER => Accuracy: 0.572829, Standard Deviation: (0.025419)
RANDOM_FOREST_CLASSIFIER => Accuracy: 0.573540, Standard Deviation: (0.025957)
RANDOM_FOREST_CLASSIFIER => Accuracy: 0.567055, Standard Deviation: (0.024370)
RANDOM_FOREST_CLASSIFIER => Accuracy: 0.564183, Standard Deviation: (0.018014)
RANDOM_FOREST_CLASSIFIER => Accuracy: 0.560593, Standard Deviation: (0.019280)
RANDOM_FOREST_CLASSIFIER => Accuracy: 0.571015, Standard Deviation: (0.022214)
RANDOM_FOREST_CLASSIFIER => Accuracy: 0.559144, Standard Deviation: (0.019413)
RANDOM_FOREST_CLASSIFIER => Accuracy: 0.570653, Standard Deviation: (0.023952)
RANDOM_FOREST_CLASSIFIER => Accuracy: 0.573530, Standard Deviation: (0.026176)
RANDOM_FOREST_CLASSIFIER => Accuracy: 0.576054, Standard Deviation: (0.040844)
Média das 10 acurácias:  0.5688596219798355

LOGISTIC_REGRESSION => Accuracy: 0.572102, Standard Deviation: (0.029217)
LOGISTIC_REGRESSION => Accuracy: 0.564537, Standard Deviation: (0.036579)
LOGISTIC_REGRESSI

'fig = plt.figure(figsize = (12,8))\nfig.suptitle("Comparação de algoritmos")\nax = fig.add_subplot(111)\nsns.barplot( x=plota_names, y=final_accuracy, data=df_lol, color="b")\nax.set_xticklabels(plota_names)\nax.set_yticklabels(final_accuracy)\nplt.show()'

<h3>Variável de interesse com maior influência nas chances de vitória</h3>
<p>Qual variável de interesse exerce maior influência no cálculo das vitórias? Quais os relacionamentos entre as variáveis de interesse? Essas respostas serão dada abaixo. </p>

In [ ]:
fields_to_correlate = ['side', 'team', 'champion1', 'position1', 'champion2', 'position2', 'champion3', 'position3', 'champion4', 'position4', 'champion5', 'position5', 'enemy_champion1', 'enemy_position1', 'enemy_champion2', 'enemy_position2', 'enemy_champion3', 'enemy_position3', 'enemy_champion4', 'enemy_position4', 'enemy_champion5', 'enemy_position5']#.split()


In [ ]:
'''
https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b

As seen from below code, the optimum number of features is 10. We now feed 10 
as number of features to RFE and get the final set of features given by RFE method, as follows:

'''


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso


#no of features
nof_list=np.arange(1,13)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

<h3> Treinando as variáveis de interesse com o modelo escolhido:</h3>
<p> Aqui, finalmente as variáveis de interesse serão de fato treinadas com o modelo preditivo que se saiu melhor na fase anterior e sua acurácia será calculada logo depois. Isso será feito para todas as variáveis de interesse ao mesmo tempo, com o objetivo de prever as chances de vitória.</p>

In [23]:
probabilites_variables = np.array(df_lol.iloc[:,1:])
target = np.array(df_lol['result'])

In [24]:
'''
Aqui, de fato o dataset está sendo treinado de acordo com a divisão feita anteriormente feita 
anteriormente entre "features" e variáveis "alvo" (result). Existem vários modelos abaixo, mas somente
o modelo que se saiu melhor durante os testes do cross_validation_score ( melhor acurácia), será 
usado. Nesse caso, a variável s_side == Lado em que o time começou na partida (Azul ou Vermelho) será
treinada.
''' 
resultados = []
n_folds = 10
k_folds = KFold(n_folds, shuffle=True, random_state = 1)
scores_teste = 0
scores_treino = 0



for linhas_treino, linhas_teste in k_folds.split(probabilites_variables,target):
    XTreino, XTeste = probabilites_variables[linhas_treino], probabilites_variables[linhas_teste]
    yTreino, yTeste = target[linhas_treino], target[linhas_teste]
    
    # Aqui o melhor modelo será escolhido para treinar as variáveis.    
    #modelr = RandomForestClassifier(n_jobs = 4, n_estimators=10, random_state = 1)
    #modelo = LogisticRegression(solver='liblinear', multi_class='ovr')
    #modeli = LinearDiscriminantAnalysis()
    #modelk = KNeighborsClassifier()
    #modeldtr = DecisionTreeRegressor(max_depth=8, min_samples_leaf=0.13, random_state=3)
    #modeld = DecisionTreeClassifier()
    #modelg = GaussianNB()
    #modelss = StandardScaler()
    models = SVC(gamma='auto')
    
    
    models.fit(XTreino, yTreino)
    previsoes = models.predict(XTeste)
    resultados.append(accuracy_score(yTeste, previsoes))
    scores_treino += models.score(XTreino, yTreino)
    scores_teste += models.score(XTeste, yTeste)
    

    
#Calcula a acurácia    
    
print("Acurácias de cada pedaço dividido pelo dataset =>")
print(resultados)
print("\nMédia de todas as acurácias: ", np.mean(resultados))
print()
print("Média da pontuação de validação do modelo de treino: ", scores_treino/n_folds)
print("Média da pontuação de validação do modelo de teste: ", scores_teste/n_folds)



Acurácias de cada pedaço dividido pelo dataset =>
[0.5016181229773463, 0.5792880258899676, 0.6407766990291263, 0.5825242718446602, 0.5728155339805825, 0.5242718446601942, 0.5728155339805825, 0.5825242718446602, 0.5533980582524272, 0.5922330097087378]

Média de todas as acurácias:  0.5702265372168285

Média da pontuação de validação do modelo de treino:  1.0
Média da pontuação de validação do modelo de teste:  0.5702265372168285
